In [1]:
from datetime import datetime
import yaml
from backbone.utils.general_purpose import load_function
from backtest.utils import plot_full_equity_curve, walk_forward
import pandas as pd

root = './backbone/data'

with open('configs/live_trading.yml', 'r') as file:
    strategies = yaml.safe_load(file)

with open('configs/test_creds.yml', 'r') as file:
    creds = yaml.safe_load(file)

date_from = datetime(2020, 1, 1)
date_to = datetime(2024, 9, 1)

not_run = [
    'backbone.eom_trader.EndOfMonthTrader',
    'backbone.vix_trader.VixTrader',    
    'backbone.b_percent_trader.BPercentTrader',
    # 'backbone.macd_trader.MacdTrader',
    'backbone.mean_reversion_trader.MeanRevTrader',
    'backbone.bbands_cross_trader.BbandsCrossTrader',
    # 'backbone.day_per_week_trader.DayPerWeekTrader'
]

equity_curves = {}
trades = {}
initial_value = 10_000


for bot_name, configs in strategies.items():

    instruments_info = configs['instruments_info']
    wfo_params = configs['wfo_params']
    opt_params = configs['opt_params']

    if bot_name in not_run:
        continue

    for ticker, info in instruments_info.items():

        timeframe = info['timeframe']
        
        name = f'{bot_name.split(".")[-1]}_{ticker}_{timeframe}'
        print(name)
        
        cron = info['cron']
        timeframe = info['timeframe']
        contract_volume = info['contract_volume']
    
        bot = load_function(bot_name)(ticker, timeframe, contract_volume, creds, opt_params, wfo_params)
        
        if bot_name == 'backbone.vix_trader.VixTrader':
            df = bot.get_full_data(date_from, date_to)
 
        else:
            df = bot.trader.get_data(date_from, date_to)
        
        
        if ticker == 'US500m' or ticker == 'USTECm' or ticker == 'US30m':
            fracc_df = df * 0.01
        else:
            fracc_df = df


        wfo_stats = walk_forward(
            bot.strategy,
            fracc_df, 
            lookback_bars=bot.wfo_params['look_back_bars'],
            validation_bars=250,
            warmup_bars=bot.wfo_params['warmup_bars'], 
            params=bot.opt_params,
            commission=7e-4, 
            margin=1/30, 
            cash=initial_value,
            verbose=True
        )
        
        equity_curves[name] = wfo_stats['_equity']
        trades[name] = wfo_stats['_trades']

MacdTrader_XPEVm_M15
  login=82847194
  trade_mode=0
  leverage=500
  limit_orders=1024
  margin_so_mode=0
  trade_allowed=True
  trade_expert=True
  margin_mode=2
  currency_digits=2
  fifo_close=False
  balance=1372.36
  credit=0.0
  profit=3.98
  equity=1376.34
  margin=40.55
  margin_free=1335.79
  margin_level=3394.1800246609127
  margin_so_call=60.0
  margin_so_so=0.0
  margin_initial=0.0
  margin_maintenance=0.0
  assets=0.0
  liabilities=0.0
  commission_blocked=0.0
  name=Said
  server=Exness-MT5Trial11
  currency=USD
  company=Exness Technologies Ltd
train from 2021-10-28 17:00:00 to 2021-12-14 15:30:00
validate from 2021-12-14 15:45:00 to 2021-12-23 11:30:00
{'cum_rsi_open_threshold': 75, 'cum_rsi_close_threshold': 55, 'risk': 1}
{'cum_rsi_open_threshold': 75, 'cum_rsi_close_threshold': 55, 'risk': 1}
equity final: 10273.996744478623
train from 2021-11-08 13:00:00 to 2021-12-23 11:30:00
validate from 2021-12-23 12:00:00 to 2022-01-04 14:00:00
{'cum_rsi_open_threshold': 75, '

In [2]:
import pandas as pd

min_date = None
max_date = None

for name, curve in equity_curves.items():
    # Convertir las fechas a UTC si son tz-naive
    actual_date = curve.index[0].tz_localize('UTC') if curve.index[0].tz is None else curve.index[0].tz_convert('UTC')
    
    # Si min_date es None, inicializar con la primera fecha
    if min_date is None:
        min_date = actual_date
    # Comparar si la fecha actual es menor que min_date
    elif actual_date < min_date:
        min_date = actual_date

    # Si max_date es None, inicializar con la última fecha
    curve_last_date = curve.index[-1].tz_localize('UTC') if curve.index[-1].tz is None else curve.index[-1].tz_convert('UTC')
    
    if max_date is None:
        max_date = curve_last_date
    # Comparar si la fecha actual es mayor que max_date
    elif curve_last_date > max_date:
        max_date = curve_last_date

# Mostrar las fechas encontradas
print(f"Min Date: {min_date}")
print(f"Max Date: {max_date}")

# Calcular min_date y max_date
min_date = min_date.date()
max_date = max_date.date()

print(min_date)
print(max_date)


date_range = pd.to_datetime(pd.date_range(start=min_date, end=max_date, freq='D'))
print(date_range)

Min Date: 2021-12-14 15:45:00+00:00
Max Date: 2024-08-29 16:15:00+00:00
2021-12-14
2024-08-29
DatetimeIndex(['2021-12-14', '2021-12-15', '2021-12-16', '2021-12-17',
               '2021-12-18', '2021-12-19', '2021-12-20', '2021-12-21',
               '2021-12-22', '2021-12-23',
               ...
               '2024-08-20', '2024-08-21', '2024-08-22', '2024-08-23',
               '2024-08-24', '2024-08-25', '2024-08-26', '2024-08-27',
               '2024-08-28', '2024-08-29'],
              dtype='datetime64[ns]', length=990, freq='D')


In [18]:

total = pd.DataFrame()
for name, curve in equity_curves.items():

    eq = equity_curves[name].copy()
    eq = eq.reset_index().rename(columns={'index':'Date'})[['Date','Equity']].sort_values(by='Date')
    eq['Date'] = pd.to_datetime(eq['Date'])
    eq['Date'] = eq['Date'].dt.floor('D')

    eq = eq.groupby('Date').agg({'Equity':'last'})

    eq = eq.reindex(date_range)
    
    eq.Equity = eq.Equity.ffill()
    eq.Equity = eq.Equity.fillna(initial_value)
    
    
    total = pd.concat([total, eq])


# total.index = pd.to_datetime(total.index, utc=True)
total = total.reset_index().rename(columns={'index':'Date'})
total = total.sort_values(by='Date')
total = total.groupby(by='Date').agg({'Equity':'sum'}).reset_index().rename(columns={'index':'Date'})

total['pct'] = (total.Equity / (initial_value * len(equity_curves.keys()))) - 1
total

,Date,Equity,pct
0,2021-12-14,30000.000000,0.000000
1,2021-12-15,29978.196581,-0.000727
2,2021-12-16,29996.817011,-0.000106
3,2021-12-17,30081.053728,0.002702
4,2021-12-18,30081.053728,0.002702
...,...,...,...
985,2024-08-25,49700.493877,0.656683
986,2024-08-26,49700.493877,0.656683
987,2024-08-27,49700.493877,0.656683
988,2024-08-28,49740.083041,0.658003


In [16]:
del equity_curves['simulated_equity_curve']

In [13]:
# initial_value = 1_000

# total = pd.DataFrame()

# for name, curve in equity_curves.items():

#     eq = equity_curves[name].copy()
#     trad = trades[name][['ExitTime', 'PnL']].copy()

#     # Renombro las columnas
#     trad = trad.rename(columns={'ExitTime':'Date'})
#     eq = eq.reset_index().rename(columns={'index':'Date'})[['Date','Equity']].sort_values(by='Date')

#     trad['Date'] = pd.to_datetime(trad['Date'])
#     eq['Date'] = pd.to_datetime(eq['Date'])

#     # joineo los dfs por fechas
#     full_df = pd.merge(
#         eq,
#         trad,
#         on='Date',
#         how='left'   
#     )

#     full_df = full_df[~full_df['PnL'].isna()]

#     full_df['Date'] = full_df['Date'].dt.floor('D')

#     full_df = full_df.groupby('Date').agg({'Equity':'last', 'PnL':'sum'})


#     full_df.fillna(0, inplace=True)
#     total = pd.concat([total, full_df])


# total.index = pd.to_datetime(total.index, utc=True)
# total = total.sort_values(by='Date')


# total = total.groupby(by='Date').agg({'Equity':'sum','PnL':'sum'}).reset_index().rename(columns={'index':'Date'})
# total = total[total.PnL != 0].copy()


# first_row = pd.DataFrame(
#     {
#         'Date':[pd.to_datetime('2021-12-14')],
#         'Equity':[initial_value],
#         'PnL':[0],
#         'pct':[0],
#     }
# )

# total = pd.concat(
#     [first_row, total]
# ).reset_index().drop(columns=['index'])

# total['pct'] = total['PnL'] / total['Equity'].shift(1)

# total = total.fillna(0)

# total


In [22]:
import numpy as np
import pandas as pd

# Calcular el comportamiento de la cartera aplicando los porcentajes de cambio
equity_curve = initial_value + (initial_value * total.pct)

# Convertir en DataFrame
equity_curve = pd.DataFrame(equity_curve)
equity_curve['Date'] = total.Date

equity_curve = equity_curve.set_index('Date').rename(columns={'pct':'Equity'})

# equity_curve = equity_curve.reindex(date_range)
equity_curve['Equity'] = equity_curve['Equity'].ffill().fillna(initial_value)

# Mostrar el resultado
equity_curve

,Equity
Date,
2021-12-14,10000.000000
2021-12-15,9992.732194
2021-12-16,9998.939004
2021-12-17,10027.017909
2021-12-18,10027.017909
...,...
2024-08-25,16566.831292
2024-08-26,16566.831292
2024-08-27,16566.831292


In [23]:
import plotly.graph_objects as go

equity_curves['simulated_equity_curve'] = equity_curve


# Crear una figura vacía
fig = go.Figure()

# Recorrer las curvas de equity de cada bot y agregarlas al gráfico
for k, v in equity_curves.items():
    
    fig.add_trace(go.Scatter(x=v.index, y=v.Equity, mode='lines', name=k))

# Actualizar los detalles del layout del gráfico
fig.update_layout(
    title="Curvas de Equity de Múltiples Bots",
    xaxis_title="Fecha",
    yaxis_title="Equity",
    legend_title="Bots"
)

# Mostrar el gráfico
fig.show()


In [28]:
trades['BPercentTrader_RLXm_M30']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-63,337,380,1.568901,1.540000,1.820763,0.018421,2022-08-04 18:30:00,2022-08-10 14:00:00,5 days 19:30:00
1,-61,396,419,1.628859,1.550000,4.810399,0.048414,2022-08-11 15:30:00,2022-08-15 14:00:00,3 days 22:30:00
2,-67,525,549,1.478964,1.500000,-1.409412,-0.014223,2022-08-25 15:00:00,2022-08-29 14:00:00,3 days 23:00:00
0,-67,275,313,1.478964,1.400000,5.290588,0.053391,2022-08-25 15:00:00,2022-08-30 14:30:00,4 days 23:30:00
1,-73,404,442,1.369041,1.280000,6.499993,0.065039,2022-09-09 14:30:00,2022-09-14 14:00:00,4 days 23:30:00
...,...,...,...,...,...,...,...,...,...,...
7,-22591,529,530,1.658838,1.660000,-26.250742,-0.000700,2024-08-27 14:00:00,2024-08-27 14:30:00,0 days 00:30:00
8,-26143,531,532,1.658838,1.640000,492.481834,0.011356,2024-08-27 15:00:00,2024-08-27 15:30:00,0 days 00:30:00
9,-25035,533,533,1.628859,1.644769,-398.310004,-0.009768,2024-08-27 16:00:00,2024-08-27 16:00:00,0 days 00:00:00
10,-22513,542,542,1.628859,1.645893,-383.480178,-0.010457,2024-08-28 14:00:00,2024-08-28 14:00:00,0 days 00:00:00


In [ ]:
equity_curves['DayPerWeekTrader_AMZNm_H4'].tail()

In [ ]:
equity_curves['DayPerWeekTrader_TSMm_H4'].tail()

In [ ]:
equity_curves['MacdTrader_XPEVm_M15'].tail()